# Нелинейные ML-модели

#### Обновлённый подход к извлечению признаков

После предзащиты было принято решение изменить стратегию формирования признаков для задач классификации направления движения цен. Обновлённый подход учитывает как специфику финансовых текстов, так и необходимость более рационального использования вычислительных ресурсов.

* **Отказ от суммаризации текста.**
В предыдущих экспериментах применялась генерация кратких аннотаций с помощью трансформеров. Однако для задачи бинарной классификации ценового движения это оказалось нецелесообразным: суммаризация может терять важные локальные сигналы, а её вычислительная стоимость высока. В новой версии анализ проводится на полном (или усечённом) оригинальном тексте без дополнительного обобщения.

* **Векторизация текста без трансформеров**
Теперь вместо BERT-подобных моделей используются TF-IDF + SVD - это ускоряет обработку в десятки раз при сохранении ключевых признаков. Метод особенно эффективен для обучения CatBoost/LightGBM на новостных текстах, хотя немного уступает трансформерам в анализе сложных контекстов.

* **Сентимент-анализ через VADER.**
Для оценки тональности текстов применяется инструмент VADER (Valence Aware Dictionary and sEntiment Reasoner) - это модель, специально настроенная на анализ настроений в социальных медиа. Несмотря на то, что VADER не предназначен специально для финансовых новостей, его простота, эффективность и открытость делают его подходящим выбором для предварительного сентимент-анализа в рамках данного исследования.

* **Увеличение объёма выборки.**
Для обучения моделей мы используем выборку из 20 тысяч строк. При этом целевая переменная - направление изменения цены за 24 часа - была сбалансирована: из исходного набора отдельно отобраны равные по размеру подвыборки с положительным и отрицательным изменением цены (по 10 тысяч примеров каждого класса). Это важно для предотвращения смещения моделей в сторону более часто встречающегося класса и повышения качества классификации.


In [12]:
# Проверка доступности CUDAimport torch
import torch

if torch.cuda.is_available():
    print("GPU is available!")
    print(f"Device Name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")

GPU is available!
Device Name: NVIDIA GeForce RTX 3080 Ti


In [16]:
# Подготовка датасета из 20 тыс. строк с равномерным распределением целевой переменной
import pandas as pd
import glob
from sklearn.utils import resample

RANDOM_SEED = 42

files = sorted(glob.glob('000[0-6].parquet'))
df = pd.concat([pd.read_parquet(f, engine='pyarrow') for f in files], ignore_index=True)

df = df.iloc[:200_000]

# Вычисляем изменение цены за 24 часа (в процентах)
df['price_24h_change_percent'] = ((df['weighted_avg_24_hrs'] - df['weighted_avg_0_hrs']) / df['weighted_avg_0_hrs'] * 100).round(2)

# Добавляем таргет: 1 — рост, 0 — падение/нулевое
df['target'] = (df['price_24h_change_percent'] > 0).astype(int)

# Балансируем данные: делаем выборку 10k примеров каждого класса
df_0 = df[df['target'] == 0]
df_1 = df[df['target'] == 1]

df_0_sample = resample(df_0, replace=False, n_samples=10_000, random_state=RANDOM_SEED)
df_1_sample = resample(df_1, replace=False, n_samples=10_000, random_state=RANDOM_SEED)

# Объединяем
df = pd.concat([df_0_sample, df_1_sample]).sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

# Проверка
print(df['target'].value_counts())

target
1    10000
0    10000
Name: count, dtype: int64


In [19]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Symbol                    20000 non-null  object 
 1   Security                  20000 non-null  object 
 2   Sector                    19618 non-null  object 
 3   Industry                  19618 non-null  object 
 4   URL                       20000 non-null  object 
 5   Date                      20000 non-null  object 
 6   RelatedStocksList         14802 non-null  object 
 7   Article                   20000 non-null  object 
 8   Title                     19967 non-null  object 
 9   articleType               20000 non-null  object 
 10  Publication               19982 non-null  object 
 11  Author                    13930 non-null  object 
 12  weighted_avg_-96_hrs      20000 non-null  float64
 13  weighted_avg_-48_hrs      20000 non-null  float64
 14  weight

,Symbol,Security,Sector,Industry,URL,Date,RelatedStocksList,Article,Title,articleType,...,weighted_avg_8_hrs,weighted_avg_12_hrs,weighted_avg_24_hrs,weighted_avg_48_hrs,weighted_avg_72_hrs,weighted_avg_96_hrs,weighted_avg_360_hrs,weighted_avg_720_hrs,price_24h_change_percent,target
0,GRNT,"Granite Ridge Resources, Inc",Energy,Oil & Gas Production,https://www.nasdaq.com/articles/stephens-co.-i...,"Sep 27, 2023 06:50 PM ET",Stocks,"Fintel reports that on September 27, 2023, Ste...",Stephens & Co. Initiates Coverage of Granite R...,News,...,6.21163,6.23576,6.29099,6.24631,6.10230,6.14635,6.03355,6.24295,1.69,1
1,HLIT,Harmonic Inc.,Technology,Radio And Television Broadcasting And Communic...,https://www.nasdaq.com/articles/motorola-msi-a...,"Dec 21, 2022 12:21 PM ET",Stocks|MSI|AUDC|TESS,"**Motorola Solutions, Inc.** [MSI](https://www...",Motorola (MSI) Avigilon Video Solution Gets JI...,News,...,13.05790,13.05830,13.17180,13.24000,13.07460,13.07690,13.21580,14.52770,-2.17,0
2,ECPG,"Encore Capital Group, Inc.",Finance,Finance Companies,https://www.nasdaq.com/articles/does-encore-ca...,"Sep 03, 2021 08:33 AM ET",Nasdaq-Listed Companies,"Some have more dollars than sense, they say, s...",Does Encore Capital Group (NASDAQ:ECPG) Deserv...,News,...,48.28000,48.28000,48.28690,48.28690,48.36290,48.36380,47.83010,50.11990,-0.45,0
3,GES,"Guess', Inc.",Consumer Discretionary,Apparel,https://www.nasdaq.com/press-release/guess-and...,"Jul 27, 2022 11:01 AM ET",None,LOS ANGELES--(BUSINESS WIRE)--\nIn collaborati...,GUESS and Homeboy Industries Announce the Upcy...,Press Release,...,18.53000,18.53000,18.31430,18.71790,18.95670,19.11950,20.65060,18.19090,-0.56,0
4,CPRX,"Catalyst Pharmaceuticals, Inc.",Health Care,Biotechnology: Pharmaceutical Preparations,https://www.nasdaq.com/articles/mid-day-market...,"Feb 17, 2016 12:02 PM ET",FOSL|Markets|SXC|CWCO,"Midway through trading Wednesday, the Dow trad...",Mid-Day Market Update: Crude Oil Surges 5%; Fo...,News,...,1.19000,1.19666,1.20550,1.19675,1.18069,1.18069,1.09705,1.12000,2.40,1


In [22]:
# Предобработка - убираем пустые значения, добавляем поле text
df = df.dropna(subset=['Article', 'weighted_avg_12_hrs'])
df['text'] = (df['Title'] + ' ' + df['Article']).fillna('')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Symbol                    20000 non-null  object 
 1   Security                  20000 non-null  object 
 2   Sector                    19618 non-null  object 
 3   Industry                  19618 non-null  object 
 4   URL                       20000 non-null  object 
 5   Date                      20000 non-null  object 
 6   RelatedStocksList         14802 non-null  object 
 7   Article                   20000 non-null  object 
 8   Title                     19967 non-null  object 
 9   articleType               20000 non-null  object 
 10  Publication               19982 non-null  object 
 11  Author                    13930 non-null  object 
 12  weighted_avg_-96_hrs      20000 non-null  float64
 13  weighted_avg_-48_hrs      20000 non-null  float64
 14  weight

In [28]:
# Делаем сентимент-анализ с VADER, добавляем в датасет
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
tqdm.pandas()

analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    if isinstance(text, str):
        return analyzer.polarity_scores(text)['compound']
    else:
        return 0.0

# Параллельная обработка
sentiments = Parallel(n_jobs=-1)(
    delayed(get_sentiment)(text) for text in tqdm(df['text'], desc="Sentiment Analysis")
)
df['sentiment'] = sentiments

Sentiment Analysis:   0%|          | 0/20000 [00:00<?, ?it/s]

In [40]:
# Подготовка данных для моделей
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
import lightgbm as lgb
from catboost import CatBoostClassifier
import numpy as np
from tqdm import tqdm

# Train/Test Split до всех трансформаций
X_text = df['text'].astype(str).str.slice(0, 300)
y = df['target'].values
X_text_train, X_text_test, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

df_train = df.loc[X_text_train.index]
df_test = df.loc[X_text_test.index]

# TF-IDF
print("TF-IDF fitting...")
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = tfidf.fit_transform(tqdm(X_text_train, desc="TF-IDF Train"))
X_test_tfidf = tfidf.transform(X_text_test)

# SVD
print("SVD fitting...")
svd = TruncatedSVD(n_components=100, random_state=RANDOM_SEED)
X_train_svd = svd.fit_transform(X_train_tfidf)
X_test_svd = svd.transform(X_test_tfidf)

# Категориальные признаки
cat_cols = ['Sector', 'Industry', 'articleType', 'Publication']
for col in cat_cols:
    df_train[col] = df_train[col].astype(str).fillna('Unknown')
    df_test[col] = df_test[col].astype(str).fillna('Unknown')

# One-hot Encoding
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_cat_train_ohe = ohe.fit_transform(df_train[cat_cols])
X_cat_test_ohe = ohe.transform(df_test[cat_cols])

# Ordinal Encoding для CatBoost с обработкой новых категорий
ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_cat_train_le = ord_enc.fit_transform(df_train[cat_cols])
X_cat_test_le = ord_enc.transform(df_test[cat_cols])

# Финальные наборы признаков
X_train_basic = np.hstack([X_cat_train_ohe, df_train[['sentiment']].values])
X_test_basic = np.hstack([X_cat_test_ohe, df_test[['sentiment']].values])

X_train_with_embeds = np.hstack([X_train_basic, X_train_svd])
X_test_with_embeds = np.hstack([X_test_basic, X_test_svd])

X_train_catboost = np.hstack([X_cat_train_le, df_train[['sentiment']].values])
X_test_catboost = np.hstack([X_cat_test_le, df_test[['sentiment']].values])

X_train_catboost_with_embeds = np.hstack([X_train_catboost, X_train_svd])
X_test_catboost_with_embeds = np.hstack([X_test_catboost, X_test_svd])

TF-IDF fitting...


TF-IDF Train: 100%|██████████| 16000/16000 [00:00<00:00, 26066.17it/s]


SVD fitting...


In [42]:
# Обучение и оценка
def train_and_evaluate(X_train, X_test, y_train, y_test, model_fn, name):
    model = model_fn()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return {
        "Model": name,
        "Accuracy": accuracy_score(y_test, preds),
        "F1 Score": f1_score(y_test, preds)
    }

results = []

# LightGBM (no embeddings)
results.append(train_and_evaluate(
    X_train_basic, X_test_basic, y_train, y_test,
    lambda: lgb.LGBMClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=RANDOM_SEED),
    "LightGBM (no embeddings)"
))

# LightGBM (with embeddings)
results.append(train_and_evaluate(
    X_train_with_embeds, X_test_with_embeds, y_train, y_test,
    lambda: lgb.LGBMClassifier(n_estimators=150, learning_rate=0.07, max_depth=8, random_state=RANDOM_SEED),
    "LightGBM (with embeddings)"
))


# CatBoost (no embeddings)
results.append(train_and_evaluate(
    X_train_catboost, X_test_catboost, y_train, y_test,
    lambda: CatBoostClassifier(verbose=0, random_seed=RANDOM_SEED, iterations=300, learning_rate=0.05, depth=6),
    "CatBoost (no embeddings)"
))

# CatBoost (with embeddings)
results.append(train_and_evaluate(
    X_train_catboost_with_embeds, X_test_catboost_with_embeds, y_train, y_test,
    lambda: CatBoostClassifier(verbose=0, random_seed=RANDOM_SEED, iterations=300, learning_rate=0.05, depth=6),
    "CatBoost (with embeddings)"
))

# GBC (no embeddings)
results.append(train_and_evaluate(
    X_train_basic, X_test_basic, y_train, y_test,
    lambda: GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=RANDOM_SEED),
    "GBC (no embeddings)"
))

# GBC (with embeddings)
results.append(train_and_evaluate(
    X_train_with_embeds, X_test_with_embeds, y_train, y_test,
    lambda: GradientBoostingClassifier(n_estimators=150, learning_rate=0.07, max_depth=8, random_state=RANDOM_SEED),
    "GBC (with embeddings)"
))

import pandas as pd
print(pd.DataFrame(results))

[LightGBM] [Info] Number of positive: 8000, number of negative: 8000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000106 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 520
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 142
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

Для эксперимента выполним подбор гиперпараметров для модели LightGBM с помощью RandomizedSearchCV. Используется пространство параметров, включающее число листьев, скорость обучения, глубину деревьев и другие важные настройки. В качестве метрики оптимизации применяется F1 score. Подбор происходит с трёхкратной кросс-валидацией и 30 случайными итерациями. После поиска лучших параметров обучается финальная модель на тренировочных данных и оценивается на тестовой выборке с помощью метрик Accuracy, F1 Score, а также строится матрица ошибок для анализа качества классификации.

In [58]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

lgbm = lgb.LGBMClassifier(random_state=RANDOM_SEED)

param_dist = {
    'num_leaves': [15, 31, 63, 127],
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 7, -1],
    'min_child_samples': [5, 10, 20],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
}

f1_scorer = make_scorer(f1_score)

search = RandomizedSearchCV(
    lgbm,
    param_distributions=param_dist,
    scoring=f1_scorer,
    n_iter=30,
    cv=3,
    verbose=1,
    random_state=RANDOM_SEED,
    n_jobs=-1
)

search.fit(X_train_with_embeds, y_train)

# Лучшая модель
best_model = search.best_estimator_
print("Лучшие параметры:", search.best_params_)

y_pred = best_model.predict(X_test_with_embeds)

# Метрики
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"Accuracy:  {accuracy:.4f}")
print(f"F1 Score:  {f1:.4f}")
print("Confusion Matrix:")
print(cm)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[LightGBM] [Info] Number of positive: 8000, number of negative: 8000
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003646 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26058
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 261
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

### Вывод по применению моделей машинного обучения

В рамках задачи бинарной классификации направления изменения цены на основе финансовых новостей были протестированы как линейные, так и нелинейные модели машинного обучения.  

#### Линейные модели (признаки на основе трансформеров):
- **LinearSVC**: Accuracy 0.50, F1 Score 0.58  
- **Logistic Regression**: Accuracy 0.52, F1 Score 0.54  

Для этих моделей признаки формировались через сложную цепочку:
1. Суммаризация текста (BART)
2. Сентимент-анализ (FinBERT)
3. Получение эмбеддингов (FinBERT)

Несмотря на применение мощных трансформеров, результаты оказались слабыми - вероятно, из-за потери важных деталей при суммаризации и высокой обобщённости признаков.

#### Нелинейные модели:

| Model                         | Accuracy | F1 Score |
|------------------------------|----------|----------|
| LightGBM (no embeddings)     | 0.54300  | 0.5289   |
| LightGBM (with embeddings)   | 0.53500  | 0.5369   |
| CatBoost (no embeddings)     | 0.53050  | 0.5162   |
| CatBoost (with embeddings)   | 0.52600  | 0.5217   |
| GBC (no embeddings)          | 0.53925  | 0.5239   |
| GBC (with embeddings)        | 0.52575  | 0.5352   |

**LightGBM с подбором гиперпараметров** показал немного лучший результат:  
Accuracy: 0.5377, F1 Score: 0.5408

**Особенности подготовки признаков:**
- **Суммаризация была исключена**, чтобы не терять значимые сигналы в тексте.
- **Векторы признаков** строились с помощью TF-IDF и SVD для простоты и эффективности.
- **Сентименты** получались с помощью лёгкой и быстрой модели **VADER**.
- **Балансировка классов** позволила избежать перекоса в сторону одного класса и улучшить устойчивость моделей.

Во всех случаях модели получились довольно низкого качества. Основная причина - **высокий уровень шума и слабая выраженность сигнала** в данных: тексты новостей редко содержат прямые указания на будущую динамику цены, особенно на коротком горизонте. Кроме того, **тематическая неоднородность** публикаций и общий характер новостей (например, обзорные или аналитические материалы без чёткой оценки) затрудняют обучение моделей.

#### Переход к глубинному обучению

Учитывая ограниченность традиционных подходов, следующим шагом будт использование **глубинных нейронных сетей**, которые позволяют работать напрямую с полными текстами и способны извлекать более сложные паттерны.